In [ ]:
# Paso 1: Importar Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import os

## Cargar los Datos

In [ ]:
# Cargar el dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar el dataset (ajustando la ruta)
data_path = "../data/WA_Fn-UseC_-Telco-Customer-Churn.csv"
try:
    df = pd.read_csv(data_path)
    print("Dataset cargado exitosamente")
    print("\nPrimeras 5 filas del dataset:")
    display(df.head())
except FileNotFoundError:
    # Si la ruta anterior falla, intentar con la ruta directa
    data_path = "data/WA_Fn-UseC_-Telco-Customer-Churn.csv"
    try:
        df = pd.read_csv(data_path)
        print("Dataset cargado exitosamente desde ruta alternativa")
        print("\nPrimeras 5 filas del dataset:")
        display(df.head())
    except FileNotFoundError:
        print("Error: No se pudo encontrar el archivo. Por favor, verifica que el archivo existe en la carpeta 'data'")
        print("Rutas intentadas:")
        print("- ../data/WA_Fn-UseC_-Telco-Customer-Churn.csv")
        print("- data/WA_Fn-UseC_-Telco-Customer-Churn.csv")

## Análisis Exploratorio de Dato

In [ ]:
# Resumen Estadístico:
print("Resumen Estadístico:")
print(df.describe())
print("\n")

In [ ]:
# Distribución de Clases:
plt.figure(figsize=(8, 6))
sns.countplot(data=df, x='Churn')
plt.title('Distribución de Churn')
plt.show()

In [ ]:
# Análisis de Variables Categóricas:
categorical_columns = df.select_dtypes(include=['object']).columns
print("\nVariables Categóricas:")
for col in categorical_columns:
    if col != 'customerID':  # Excluimos el ID del cliente
        print(f"\nDistribución de {col}:")
        print(df[col].value_counts())

        # Visualización
        plt.figure(figsize=(10, 6))
        sns.countplot(data=df, x=col, hue='Churn')
        plt.title(f'Distribución de {col} por Churn')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

In [ ]:
# Correlación entre Variables Numéricas:
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
corr_matrix = df[numeric_columns].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt='.2f')
plt.title("Matriz de correlación de variables numéricas")
plt.tight_layout()
plt.show()

In [ ]:
# Histograma de Ingresos:
plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='MonthlyCharges', bins=30, kde=True)
plt.title('Distribución de MonthlyCharges')
plt.xlabel('MonthlyCharges')
plt.ylabel('Frecuencia')
plt.show()

# Información adicional sobre el dataset
print("\nInformación del Dataset:")
print(df.info())

# Valores faltantes
print("\nValores faltantes por columna:")
print(df.isnull().sum())

## Preparación de Datos

Normalicé las columnas numéricas (tenure, MonthlyCharges, y TotalCharges) utilizando StandardScaler de scikit-learn, que estandariza los datos para que tengan una media de 0 y una desviación estándar de 1.

**Justificación:**

* Usamos StandardScaler porque KNN es sensible a las escalas de las variables.
* Rellenamos valores faltantes en TotalCharges con la mediana para evitar problemas durante la normalización.
* Solo normalizamos las columnas numéricas, ya que las categóricas serán codificadas más adelante.

In [16]:
## Normalización:
from sklearn.preprocessing import StandardScaler

# Convertir 'TotalCharges' a numérico (puede contener valores no numéricos)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Rellenar valores faltantes en 'TotalCharges' con la mediana
df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)

# Seleccionar columnas numéricas para normalizar
numeric_columns = ['tenure', 'MonthlyCharges', 'TotalCharges']

# Crear un objeto StandardScaler
scaler = StandardScaler()

# Normalizar las columnas numéricas
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

print("Datos normalizados:")
print(df[numeric_columns].head())

Datos normalizados:
     tenure  MonthlyCharges  TotalCharges
0 -1.277445       -1.160323     -0.994242
1  0.066327       -0.259629     -0.173244
2 -1.236724       -0.362660     -0.959674
3  0.514251       -0.746535     -0.194766
4 -1.236724        0.197365     -0.940470


/tmp/ipykernel_2823/3593941153.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)


In [17]:
# División de Datos:
from sklearn.model_selection import train_test_split

# Codificar variables categóricas con one-hot encoding
df_encoded = pd.get_dummies(df, columns=df.select_dtypes(include=['object']).columns, drop_first=True)

# Separar características (X) y variable objetivo (y)
X = df_encoded.drop('Churn_Yes', axis=1)
y = df_encoded['Churn_Yes']

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)

Tamaño del conjunto de entrenamiento: (5634, 7072)
Tamaño del conjunto de prueba: (1409, 7072)


## Implementación de KNN

In [ ]:
# Entrenamiento del Modelo:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

In [ ]:
# Evaluación:
from sklearn.metrics import accuracy_score
y_pred = knn.predict(X_test)
print("Precisión del modelo:", accuracy_score(y_test, y_pred))

In [ ]:
# Visualización de Precisión para Diferentes Vecinos:
accuracies = []
for k in range(1, 21):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracies.append(accuracy_score(y_test, y_pred))

plt.plot(range(1, 21), accuracies)
plt.title("Precisión vs Número de Vecinos")
plt.xlabel("Número de Vecinos")
plt.ylabel("Precisión")
plt.show()